## Importing Libraries

In [66]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Load the dataset

In [67]:
df= pd.read_csv(r"C:\Users\sanoj\Downloads\dynamic_pricing (1).csv")
df.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422


In [68]:
df.describe()

,Number_of_Riders,Number_of_Drivers,Number_of_Past_Rides,Average_Ratings,Expected_Ride_Duration,Historical_Cost_of_Ride
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000
mean,60.372000,27.076000,50.031000,4.257220,99.58800,372.502623
std,23.701506,19.068346,29.313774,0.435781,49.16545,187.158756
min,20.000000,5.000000,0.000000,3.500000,10.00000,25.993449
25%,40.000000,11.000000,25.000000,3.870000,59.75000,221.365202
50%,60.000000,22.000000,51.000000,4.270000,102.00000,362.019426
75%,81.000000,38.000000,75.000000,4.632500,143.00000,510.497504
max,100.000000,89.000000,100.000000,5.000000,180.00000,836.116419


# Ratio Based Approach

## Calculating Demand_Supply_Ratio and picking demand_supply_threshold = 2.3 arround the mean of Demand_Supply_Ratio
* ### Higher Demand = when 'Demand_Supply_Ratio' > demand_supply_threshold (2.3) else Low-demand
* ### Higher supply = when 'Demand_Supply_Ratio' < demand_supply_threshold (2.3) else Low-supply



In [69]:
df['Demand_Supply_Ratio'] = df['Number_of_Riders'] / df['Number_of_Drivers']
demand_supply_threshold = 2.3

df['Demand_class'] = np.where(df['Demand_Supply_Ratio'] > demand_supply_threshold, "Higher_demand", "Lower_demand")
df['Supply_class'] = np.where(df['Demand_Supply_Ratio'] < demand_supply_threshold, "Higher_supply", "Lower_supply")

df.iloc[:,[0,1,10,11,12]].sample(10)

,Number_of_Riders,Number_of_Drivers,Demand_Supply_Ratio,Demand_class,Supply_class
655,29,7,4.142857,Higher_demand,Lower_supply
900,61,35,1.742857,Lower_demand,Higher_supply
767,46,15,3.066667,Higher_demand,Lower_supply
732,36,16,2.250000,Lower_demand,Higher_supply
184,97,18,5.388889,Higher_demand,Lower_supply
420,40,30,1.333333,Lower_demand,Higher_supply
302,25,7,3.571429,Higher_demand,Lower_supply
995,33,23,1.434783,Lower_demand,Higher_supply
581,68,29,2.344828,Higher_demand,Lower_supply
548,59,40,1.475000,Lower_demand,Higher_supply


## calulation Base Price and Surge_charge based on supply demand ratio and demand_supply_factor
* ### 1. Calculate base historical cost based on expected_Ride_duration
* ### 2. Calculate rider-to-driver ratio
* ### 3. Calculate demand-supply factor
* ### 4. Defining a methode to Calculate supply_demand_surge and Apply the dynamic pricing formula

In [70]:

constant_rate = 3.5  # Define the base rate per unit of duration, this is arround mean of ratio of ('Historical_Cost_of_Ride'/'Expected_Ride_Duration')
demand_hike = 0.35  # This is how much demand increase the pricing

# Calculate base historical cost based on expected_Ride_duration
df['base_cost'] = df['Expected_Ride_Duration'] * constant_rate

# Calculate rider-to-driver ratio
df['rider_driver_ratio'] = df['Number_of_Riders'] / df['Number_of_Drivers']

# Calculate demand-supply factor
df['demand_supply_factor'] = df['rider_driver_ratio'] - 1
df['demand_supply_factor'] = df['demand_supply_factor'].apply(lambda x: min(x, 6))

# defining a methode to Calculate supply_demand_surge and Apply the dynamic pricing formula
def apply_surge(df):
    SD_surge_charge=0
    if (df['Demand_class']=='Higher_demand') & (df['Supply_class']=='Lower_supply'):
        SD_surge_charge = df['base_cost'] * (demand_hike * df['demand_supply_factor'])
    return SD_surge_charge

df['S/D_surge_charge'] = df.apply(apply_surge,axis=1)
df.head()

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,rider_driver_ratio,demand_supply_factor,S/D_surge_charge
0,90,45,Urban,Silver,13,4.47,Night,Premium,90,284.257273,2.000000,Lower_demand,Higher_supply,315.0,2.000000,1.000000,0.000000
1,58,39,Suburban,Silver,72,4.06,Evening,Economy,43,173.874753,1.487179,Lower_demand,Higher_supply,150.5,1.487179,0.487179,0.000000
2,42,31,Rural,Silver,0,3.99,Afternoon,Premium,76,329.795469,1.354839,Lower_demand,Higher_supply,266.0,1.354839,0.354839,0.000000
3,89,28,Rural,Regular,67,4.31,Afternoon,Premium,134,470.201232,3.178571,Higher_demand,Lower_supply,469.0,3.178571,2.178571,357.612500
4,78,22,Rural,Regular,74,3.77,Afternoon,Economy,149,579.681422,3.545455,Higher_demand,Lower_supply,521.5,3.545455,2.545455,464.609091


## Conditional Surge based on Vehical_Type and Time_of_booking && Location_Category Condition


In [71]:

def cal_surge_charge(df):
    surge_charge = 0
    if df['Vehicle_Type'] == 'Premium':
        if (df['Location_Category'] in ('Urban', 'Suburban')) & (df['Time_of_Booking'] in ('Evening', 'Night')):
            surge_charge = df['base_cost'] * 0.05 + df['base_cost'] * 0.02
    else:
        if (df['Location_Category'] in ('Urban', 'Suburban')) & (df['Time_of_Booking'] in ('Evening', 'Night')):
            surge_charge = df['base_cost'] * 0.025 + df['base_cost'] * 0.01
    return surge_charge

df['Surge_charge'] = df.apply(cal_surge_charge, axis=1)


In [72]:
df.sample(10)

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,rider_driver_ratio,demand_supply_factor,S/D_surge_charge,Surge_charge
227,53,40,Rural,Gold,79,3.66,Evening,Premium,30,158.652642,1.325000,Lower_demand,Higher_supply,105.0,1.325000,0.325000,0.000000,0.000
543,100,54,Suburban,Regular,91,4.59,Night,Premium,61,203.113296,1.851852,Lower_demand,Higher_supply,213.5,1.851852,0.851852,0.000000,14.945
910,82,20,Urban,Regular,59,3.99,Night,Premium,45,241.825956,4.100000,Higher_demand,Lower_supply,157.5,4.100000,3.100000,170.887500,11.025
996,84,29,Urban,Regular,92,4.55,Morning,Premium,94,424.155987,2.896552,Higher_demand,Lower_supply,329.0,2.896552,1.896552,218.387931,0.000
292,90,56,Suburban,Silver,61,4.38,Night,Premium,91,410.342709,1.607143,Lower_demand,Higher_supply,318.5,1.607143,0.607143,0.000000,22.295
450,49,6,Suburban,Gold,18,4.41,Night,Premium,148,443.775020,8.166667,Higher_demand,Lower_supply,518.0,8.166667,6.000000,1087.800000,36.260
499,44,28,Rural,Gold,30,3.93,Evening,Premium,42,157.106572,1.571429,Lower_demand,Higher_supply,147.0,1.571429,0.571429,0.000000,0.000
37,25,15,Rural,Gold,65,4.88,Evening,Premium,101,502.529321,1.666667,Lower_demand,Higher_supply,353.5,1.666667,0.666667,0.000000,0.000
754,44,25,Rural,Silver,32,4.43,Night,Premium,154,502.101206,1.760000,Lower_demand,Higher_supply,539.0,1.760000,0.760000,0.000000,0.000
925,73,27,Urban,Regular,55,4.27,Morning,Premium,50,216.060196,2.703704,Higher_demand,Lower_supply,175.0,2.703704,1.703704,104.351852,0.000


## Calculating Total cost

In [73]:
df['New_cost']= df['base_cost'] + df['S/D_surge_charge'] + df['Surge_charge']
df.iloc[:,[0,1,9,10,11,12,13,16,17,18]].sample(10)

,Number_of_Riders,Number_of_Drivers,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,S/D_surge_charge,Surge_charge,New_cost
834,38,8,291.107937,4.750000,Higher_demand,Lower_supply,262.5,344.531250,18.375,625.406250
764,40,6,707.855393,6.666667,Higher_demand,Lower_supply,518.0,1027.366667,0.000,1545.366667
520,28,5,425.488735,5.600000,Higher_demand,Lower_supply,462.0,743.820000,0.000,1205.820000
740,22,7,552.269375,3.142857,Higher_demand,Lower_supply,630.0,472.500000,44.100,1146.600000
462,44,11,104.752010,4.000000,Higher_demand,Lower_supply,63.0,66.150000,0.000,129.150000
713,99,22,131.443587,4.500000,Higher_demand,Lower_supply,136.5,167.212500,0.000,303.712500
894,49,29,468.306920,1.689655,Lower_demand,Higher_supply,560.0,0.000000,39.200,599.200000
561,88,16,127.331490,5.500000,Higher_demand,Lower_supply,112.0,176.400000,0.000,288.400000
871,42,19,539.731672,2.210526,Lower_demand,Higher_supply,504.0,0.000000,0.000,504.000000
603,89,74,432.162792,1.202703,Lower_demand,Higher_supply,402.5,0.000000,0.000,402.500000


## Revenue Before and after

In [79]:
print("Revenue before applying Dynamic_pricing -->",round(sum(df['Historical_Cost_of_Ride']),2))
print("Revenue after applying Dynamic_pricing-->",round(sum(df['New_cost']),2))

Revenue before applying Dynamic_pricing --> 372502.62
Revenue after applying Dynamic_pricing--> 552298.01


In [80]:
diff=sum(df['New_cost'])-sum(df['Historical_Cost_of_Ride'])
print("Diffrenece of Revenue--> ", diff)
print("Revenue Percentage --> ", diff/sum(df['Historical_Cost_of_Ride'])*100)

Diffrenece of Revenue-->  179795.39090132003
Revenue Percentage -->  48.26687911203326


## Conclusion
* ### Diffrenece of Revenue-->  179795.39
* ### Revenue Percentage -->  48.26

In [81]:
filter=df['Demand_Supply_Ratio']>10
df[filter].head(10)

,Number_of_Riders,Number_of_Drivers,Location_Category,Customer_Loyalty_Status,Number_of_Past_Rides,Average_Ratings,Time_of_Booking,Vehicle_Type,Expected_Ride_Duration,Historical_Cost_of_Ride,Demand_Supply_Ratio,Demand_class,Supply_class,base_cost,rider_driver_ratio,demand_supply_factor,S/D_surge_charge,Surge_charge,New_cost
49,67,6,Rural,Gold,15,3.53,Night,Economy,123,420.623911,11.166667,Higher_demand,Lower_supply,430.5,11.166667,6.0,904.05,0.0000,1334.5500
88,66,6,Rural,Regular,23,4.20,Evening,Economy,45,173.157754,11.000000,Higher_demand,Lower_supply,157.5,11.000000,6.0,330.75,0.0000,488.2500
94,95,7,Rural,Gold,40,4.68,Evening,Economy,95,283.466443,13.571429,Higher_demand,Lower_supply,332.5,13.571429,6.0,698.25,0.0000,1030.7500
153,51,5,Urban,Gold,0,4.59,Afternoon,Premium,92,320.857622,10.200000,Higher_demand,Lower_supply,322.0,10.200000,6.0,676.20,0.0000,998.2000
170,76,7,Urban,Gold,76,4.35,Morning,Economy,72,245.893571,10.857143,Higher_demand,Lower_supply,252.0,10.857143,6.0,529.20,0.0000,781.2000
197,75,7,Suburban,Gold,100,4.13,Morning,Economy,134,453.376949,10.714286,Higher_demand,Lower_supply,469.0,10.714286,6.0,984.90,0.0000,1453.9000
216,88,5,Urban,Silver,89,3.59,Night,Economy,27,70.203803,17.600000,Higher_demand,Lower_supply,94.5,17.600000,6.0,198.45,3.3075,296.2575
218,65,5,Rural,Silver,24,3.54,Night,Economy,119,301.403927,13.000000,Higher_demand,Lower_supply,416.5,13.000000,6.0,874.65,0.0000,1291.1500
232,87,5,Urban,Silver,59,4.32,Night,Economy,42,151.359301,17.400000,Higher_demand,Lower_supply,147.0,17.400000,6.0,308.70,5.1450,460.8450
250,97,7,Urban,Silver,22,3.74,Afternoon,Premium,147,441.746701,13.857143,Higher_demand,Lower_supply,514.5,13.857143,6.0,1080.45,0.0000,1594.9500
